## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_1 = pd.read_csv("../data/gcc8/o0123합본/o1/o1_gcc8.csv", index_col=0)
core8_1 = pd.read_csv("../data/gcc8/o0123합본/o1/o1_core8.csv", index_col=0)
print(bin8_1.shape, core8_1.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_1.reset_index(inplace=True, drop=True)
core8_1.reset_index(inplace=True, drop=True)

print('bin - core concat')
bin8_1 = pd.concat([bin8_1, core8_1], axis=0)
print('shape -', bin8_1.shape)
print('reset_index 완료')
print('input data shape')
bin8_1.head()

(20380454, 2) (7286847, 2)
bin - core concat
shape - (27667301, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_1['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_1['label'].value_counts())

256
0    27628838
1       38463
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_1[bin8_1['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_1), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_1', len(ls_idx_bin))

# loc 로 수정필요
bin8_1_Ngram = bin8_1.loc[ls_idx_bin,:].copy()

1230816
나머지 0
최종 길이 1230816
bin8_1 1230816


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_1_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_1.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_1)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_1[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

60531.0
0 19424608
1000 5093078
2000 17699137
3000 6751742
4000 27152945
5000 27522718
6000 20337190
7000 25416428
8000 8021659
9000 2548471
10000 8336910
11000 21314356
12000 18938224
13000 18910524
14000 4491786
15000 11317485
16000 19742431
17000 5974617
18000 881873
19000 7239362
20000 27072766
21000 21334776
22000 15372939
23000 14298245
24000 5201653
25000 10882994
26000 7189866
27000 27274700
28000 25541856
29000 25864857
30000 21588013
31000 16197718
32000 9920594
33000 21507437
34000 4687342
35000 22762190
36000 372260
37000 12427885
38000 27284242
39000 16160669
40000 14245228
41000 15148240
42000 21725610
43000 13523464
44000 9316936
45000 16064795
46000 23612339
47000 1311782
48000 19021458
49000 19947481
50000 20534070
51000 26794895
52000 16939254
53000 27635452
54000 7226228
55000 25958695
56000 5281839
57000 27447494
58000 108216
59000 16979976
60000 15254769
완료
60531


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_1['label'].value_counts()[1])

60531
38463


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_1_Ngram])
final.shape

(3873984, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_1_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_1_onehot_Ngram = pd.concat([final['label'], bc8_1_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_1_onehot_Ngram.shape)

원핫인코딩완료
(3873984, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_1 = bc8_1_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_1 = bc8_1_onehot_Ngram['label'].to_numpy()
print(x_bc8_1.shape, x_bc8_1.shape)

x_bc8_1 = x_bc8_1.reshape(-1, right_idx, x_bc8_1.shape[1])
y_bc8_1 = y_bc8_1.reshape(-1, right_idx, 1)

print(x_bc8_1.shape, y_bc8_1.shape)

(3873984, 256) (3873984, 256)
(121062, 32, 256) (121062, 32, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_1.shape[0])

x_bc8_1 = x_bc8_1[p]
y_bc8_1 = y_bc8_1[p]

print(x_bc8_1.shape, y_bc8_1.shape)

(121062, 32, 256) (121062, 32, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_1, y_bc8_1):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_1[train],
               y_bc8_1[train],
               epochs = 10,
               batch_size = 128,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_1[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_1[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108955 samples
Epoch 1/10
108955/108955 [==============================] - 10s 96us/sample - loss: 0.0281 - accuracy: 0.9923
Epoch 2/10
108955/108955 [==============================] - 6s 57us/sample - loss: 0.0074 - accuracy: 0.9977
Epoch 3/10
108955/108955 [==============================] - 6s 58us/sample - loss: 0.0055 - accuracy: 0.9983
Epoch 

accuracy_score 0.9991688692491947
recall_score 0.9649744195198741
precision_score 0.971473851030111
f1_score 0.968213228035538
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108955 samples
Epoch 1/10
108955/108955 [==============================] - 9s 86us/sample - loss: 0.0017 - accuracy: 0.9995
Epoch 2/10
108955/108955 [==============================] - 6s 58us/sample - loss: 0.0015 - accu

accuracy_score 0.9994889320227968
recall_score 0.980944819372767
precision_score 0.9797779540047581
f1_score 0.9803610394762943
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 9s 81us/sample - loss: 6.7033e-04 - accuracy: 0.9998
Epoch 2/10
108956/108956 [==============================] - 6s 58us/sample - loss: 5.4931e-

accuracy_score 0.9997496076325789
recall_score 0.9889785475300138
precision_score 0.9919068298460324
f1_score 0.9904405242928945
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 9s 82us/sample - loss: 2.7484e-04 - accuracy: 0.9999
Epoch 2/10
108956/108956 [==============================] - 6s 58us/sample - loss: 2.0884e

accuracy_score 0.9998425367586321
recall_score 0.9935897435897436
precision_score 0.9945557067859226
f1_score 0.994072490525702
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 9s 82us/sample - loss: 1.6162e-04 - accuracy: 1.0000
Epoch 2/10
108956/108956 [==============================] - 6s 59us/sample - loss: 8.9135e-

...

accuracy_score 0.9999638608954238
recall_score 0.9976589933671479
precision_score 0.9996090695856138
f1_score 0.9986330794766648
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 9s 82us/sample - loss: 3.3725e-05 - accuracy: 1.0000
Epoch 2/10
108956/108956 [==============================] - 6s 59us/sample - loss: 2.7678e

accuracy_score 0.9999690236246489
recall_score 0.9982089552238806
precision_score 0.9994022713687986
f1_score 0.998805256869773
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 10s 95us/sample - loss: 2.9325e-05 - accuracy: 1.0000
Epoch 2/10
108956/108956 [==============================] - 6s 59us/sample - loss: 2.0858e

accuracy_score 0.9999974186353874
recall_score 0.9997960432388334
precision_score 1.0
f1_score 0.999898011218766
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 9s 82us/sample - loss: 1.3594e-05 - accuracy: 1.0000
Epoch 2/10
108956/108956 [==============================] - 6s 59us/sample - loss: 1.8675e-05 - accuracy: 

accuracy_score 0.9999948372707749
recall_score 0.9997980206018986
precision_score 0.9997980206018986
f1_score 0.9997980206018986
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 108956 samples
Epoch 1/10
108956/108956 [==============================] - 9s 82us/sample - loss: 1.2452e-05 - accuracy: 1.0000
Epoch 2/10
108956/108956 [==============================] - 6s 60us/sample - loss: 9.3109e

accuracy_score 0.9999922559061623
recall_score 0.9998055987558321
precision_score 0.9996112730806609
f1_score 0.9997084264748761

K-fold cross validation Accuracy: [0.9991688692491947, 0.9994889320227968, 0.9997496076325789, 0.9998425367586321, 0.9999251404262349, 0.9999638608954238, 0.9999690236246489, 0.9999974186353874, 0.9999948372707749, 0.9999922559061623]

K-fold cross validation Recall: [0.9649744195198741, 0.980944819372767, 0.9889785475300138, 0.9935897435897436, 0.9965580076938652, 0.9976589933671479, 0.9982089552238806, 0.9997960432388334, 0.9997980206018986, 0.9998055987558321]

K-fold cross validation Precision: [0.971473851030111, 0.9797779540047581, 0.9919068298460324, 0.9945557067859226, 0.9975678962302391, 0.9996090695856138, 0.9994022713687986, 1.0, 0.9997980206018986, 0.9996112730806609]

K-fold cross validation F1-Score: [0.968213228035538, 0.9803610394762943, 0.9904405242928945, 0.994072490525702, 0.9970626962422768, 0.9986330794766648, 0.998805256869773, 0.999898

## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9998092482421835
10-Fold Cross_validation. Recall : 0.9920313148893858
10-Fold Cross_validation. Precision : 0.9933702872534035
10-Fold Cross_validation. F1-Score : 0.9926992773214686


In [13]:
model1.save('gcc8_bin_core_s32_h48_o1.h5')
print('save 완료')

save 완료
